# Laboratorio 1: Setup y uso básico de LLMs con LangChain + Prompt Engineering Avanzado

Este laboratorio está pensado para completarse en ~2 horas. Trabajarás en Google Colab con recursos gratuitos, usando la Inference API de Hugging Face y un modelo instruct abierto.

**Contenidos:**
- Setup en Colab y configuración de Hugging Face Inference API
- Uso básico de LLMs con LangChain (LCEL)
- Parámetros de decodificación y control de estilo
- Prompt engineering avanzado: zero-shot, few-shot, Chain of Thought, Role Prompting y salida estructurada (JSON)

**Al finalizar, podrás:**
- Conectarte a un LLM instruct vía Hugging Face Inference API desde LangChain
- Construir cadenas simples con `prompt | llm | parser`
- Diseñar prompts efectivos y controlar formato de salida (incluido JSON)

## Parte 0: Setup (Colab + librerías + token)

Usarás versiones estables para minimizar fricción en Colab. Asegúrate de ejecutar esta sección antes de continuar.

In [ ]:
# Instalar dependencias principales (versiones estables)
!pip -q install -U \
  "langchain==0.3.27" \
  "langchain-community==0.3.27" \
  "langchain-huggingface==0.3.1" \
  "transformers==4.55.2" \
  "huggingface_hub==0.34.4"

In [ ]:
from importlib.metadata import version, PackageNotFoundError

for dist in ["langchain", "langchain-community", "langchain-huggingface",
             "transformers", "huggingface_hub"]:
    try:
        print(dist, "=>", version(dist))
    except PackageNotFoundError:
        print(dist, "no instalado")

In [ ]:
# Comprobar versión de Python y GPU/CPU
import sys, subprocess, torch
print(sys.version)
try:
    import torch
    print("PyTorch:", torch.__version__)
    print("CUDA disponible:", torch.cuda.is_available())
except Exception as e:
    print("PyTorch no disponible o sin CUDA", e)


### Configuración del token de Hugging Face

Para usar la Hugging Face Inference API necesitás un token personal (gratuito). En Colab se recomienda guardarlo en `userdata`:

1. Crear el token en https://huggingface.co/settings/tokens
2. En Colab: Abre el menú en la barra izquierda haciendo clic en la llave → "Agregar nuevo secreto", ponle `HF_TOKEN` y el token que generaste → Habilita el acceso al notebook
3. Ejecutar la celda siguiente para leerlo

In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')
assert HF_TOKEN is not None and len(HF_TOKEN) > 0, "Configurar el secreto 'HF_TOKEN' en Colab."
print("Token cargado OK")


## Parte 1: Uso básico de LLMs con LangChain

Trabajarás con un modelo instruct accesible vía Inference API. Para minimizar fricción, usarás un modelo abierto.

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

MODEL_ID = "Qwen/Qwen3-4B-Instruct-2507"

hf_endpoint = HuggingFaceEndpoint(
    repo_id=MODEL_ID,
    task="conversational",
    huggingfacehub_api_token=HF_TOKEN,
    temperature=0.7,
    max_new_tokens=256,
)

llm = ChatHuggingFace(llm=hf_endpoint)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente útil y conciso."),
    ("human", "Responde a la siguiente instrucción: {instruccion}"),
])

chain = prompt | llm | StrOutputParser()
print(chain.invoke({"instruccion": "Explica en 3 frases qué es un LLM y nombra 2 casos de uso."}))

### Ejercicio 1.1 (10 min)

Realiza las siguientes tareas:
- Probar 3 variaciones de `temperature` y observar el cambio en estilo
- Cambiar el rol del `system` para forzar un estilo (por ejemplo: "responde con viñetas y máximo 3 líneas")
- Pregunta sugerida: "Resume la diferencia entre entrenamiento y fine-tuning en 3 puntos"

In [ ]:
from typing import List
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

instruccion: str = "Resume la diferencia entre entrenamiento y fine-tuning en 3 puntos."
temperaturas: List[float] = [0.0, 0.7, 1.2]

def ejecutar_variacion(temperature: float) -> str:
    # TODO: crear endpoint conversacional con 'temperature' y devolver el texto
    # Debe usar: MODEL_ID, HF_TOKEN, task="conversational"
    tmp_endpoint = HuggingFaceEndpoint(
        repo_id=MODEL_ID,
        task="conversational",
        huggingfacehub_api_token=HF_TOKEN,
        temperature=temperature,
        max_new_tokens=256,
    )
    tmp_llm = ChatHuggingFace(llm=tmp_endpoint)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Eres un asistente útil y conciso."),
        ("human", "Responde a la siguiente instrucción: {instruccion}"),
    ])
    chain = prompt | tmp_llm | StrOutputParser()
    return chain.invoke({"instruccion": instruccion})


for t in temperaturas:
    print(f"\n==== temperature: {t} ====")
    print(ejecutar_variacion(t))

def ejecutar_estilo(instruccion: str) -> str:
    # TODO: crear prompt de estilo (system) + LLM base conversacional y devolver el texto
    styled_prompt = ChatPromptTemplate.from_messages([
        ("system", "Responde con viñetas y máximo 3 líneas."),
        ("human", "Responde a la siguiente instrucción: {instruccion}"),
    ])
    chain = styled_prompt | llm | StrOutputParser()
    return chain.invoke({"instruccion": instruccion})


print("\n==== estilo forzado ====")
print(ejecutar_estilo(instruccion))

### Parámetros de decodificación

Ajustarás parámetros como `top_p`, `repetition_penalty` y `max_new_tokens` para observar su efecto en la generación.

In [ ]:
# Exploración de parámetros de decodificación
from typing import Dict, Any
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.output_parsers import StrOutputParser

consulta: str = "Escribe una analogía breve para explicar RAG a un público no técnico."

configuraciones: Dict[str, Dict[str, Any]] = {
    "baseline":   {"temperature": 0.7, "top_p": 0.95, "repetition_penalty": 1.0, "max_new_tokens": 128},
    "creativo":   {"temperature": 1.1, "top_p": 0.90, "repetition_penalty": 1.0, "max_new_tokens": 128},
    "controlado": {"temperature": 0.2, "top_p": 0.80, "repetition_penalty": 1.1, "max_new_tokens": 96},
}

for nombre, cfg in configuraciones.items():
    tmp_endpoint = HuggingFaceEndpoint(
        repo_id=MODEL_ID,
        task="conversational",
        huggingfacehub_api_token=HF_TOKEN,
        temperature=cfg["temperature"],
        top_p=cfg["top_p"],
        repetition_penalty=cfg["repetition_penalty"],
        max_new_tokens=cfg["max_new_tokens"],
    )
    tmp_llm = ChatHuggingFace(llm=tmp_endpoint)
    salida = (prompt | tmp_llm | StrOutputParser()).invoke({"instruccion": consulta})
    print(f"\n==== {nombre} ({cfg}) ====\n{salida}")

### Análisis de las configuraciones

#### 1. Baseline (Equilibrado)
- `temperature: 0.7` (Moderada)
- `top_p: 0.95` (Alto)
- `max_new_tokens: 128` (Largo estándar)

Esta configuración busca un **equilibrio entre coherencia y creatividad**.
- La **temperatura (0.7)** es moderada, permitiendo que el modelo sea un poco creativo sin desviarse demasiado de las respuestas más probables o "seguras"
- El **top_p (0.95)** le da un amplio "menú" de palabras posibles para elegir, fomentando la variedad

**Resultado:** La respuesta es clara, usa una buena analogía ("asistente que busca la mejor información en tiempo real") y es informativa. Es la respuesta estándar y más fiable.

---

#### 2. Creativo (Exploratorio)
- `temperature: 1.1` (Alta)
- `top_p: 0.9` (Alto)
- `max_new_tokens: 128` (Largo estándar)

El factor clave aquí es la **alta temperatura (1.1)**.
- Una temperatura superior a 1.0 **aumenta la aleatoriedad**. Fuerza al modelo a considerar palabras menos probables, lo que puede llevar a respuestas más originales o inesperadas
- El `top_p` sigue siendo alto, dándole muchas opciones

**Resultado:** La analogía es diferente y más específica que la del baseline. En lugar de un "libro de conocimientos" genérico, crea un escenario más vívido: "un libro enorme con millones de páginas", "encontrar el nombre de un personaje" y "preguntas a alguien que conoce bien el contenido". Es más narrativo y menos directo.

---

#### 3. Controlado (Conciso y Determinado)
- `temperature: 0.2` (Muy baja)
- `top_p: 0.8` (Bajo)
- `repetition_penalty: 1.1` (Ligera penalización)
- `max_new_tokens: 96` (Corto)

Esta configuración está diseñada para ser **precisa, predecible y concisa**.
- La **temperatura (0.2)** y el **top_p (0.8)** son bajos. Esto "enfría" al modelo, obligándolo a elegir casi siempre la palabra *más probable* y limitando su "menú" de opciones. El resultado es menos creativo pero muy enfocado y coherente
- La **penalización por repetición (1.1)** evita que se quede atascado repitiendo las mismas palabras "seguras" una y otra vez
- Los **tokens máximos (96)** fuerzan una respuesta más corta

**Resultado:** La respuesta es la más directa y "seca". Va al grano ("busca información relevante... y luego usa esa información"), usa un ejemplo concreto ("¿Qué sabes sobre los efectos...?") y es notablemente más corta que las otras dos.

## Parte 2: Prompt Engineering Avanzado

Explorarás estrategias para mejorar la calidad y control de las respuestas: zero-shot, few-shot, restricciones de estilo, salida estructurada y Chain of Thought.

### Conceptos clave: Zero-shot, Few-shot, CoT y Roles

- **Zero-shot**: el modelo resuelve la tarea solo con instrucciones; no se proporcionan ejemplos
- **Few-shot**: además de la instrucción, se incluyen 1-3 ejemplos que muestran el formato y estilo deseados
- **Chain-of-Thought (CoT)**: se guía al modelo para mostrar pasos intermedios de razonamiento (por ejemplo, "razona paso a paso") antes de una respuesta final
- **Role prompting**: se asigna un rol (por ejemplo, "actúa como profesor de IA") para influir en el estilo y nivel de detalle
- **JSON output**: se pide una salida estricta en formato JSON y se valida con un parser

**Lectura recomendada**: guía de prompt engineering avanzada en https://learnprompting.org/docs/introduction

### Zero-shot y Few-shot

Compararás la efectividad de proporcionar ejemplos versus no proporcionarlos.

In [ ]:
# Zero-shot vs Few-shot (diferencia marcada con patrón acróstico)
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# LLM más determinista
det_endpoint = HuggingFaceEndpoint(
    repo_id=MODEL_ID, task="conversational",
    huggingfacehub_api_token=HF_TOKEN, temperature=0.0, max_new_tokens=128
)
llm_det = ChatHuggingFace(llm=det_endpoint)
parser = StrOutputParser()

# Patrón no obvio: acróstico O-V-E (Overfitting), cada línea empieza con esa letra
instruccion = (
    "Escribe EXACTAMENTE 3 líneas sobre 'overfitting'. "
    "Cada línea debe comenzar con O, luego V, luego E (en ese orden). "
    "6-10 palabras por línea. Sin texto extra."
)

zero_shot = ChatPromptTemplate.from_messages([
    ("system", "Sigue estrictamente las instrucciones del usuario."),
    ("human", "{instruccion}")
])

few_shot = ChatPromptTemplate.from_messages([
    ("system", "Sigue estrictamente las instrucciones del usuario."),
    # Ejemplo: el patrón se demuestra con otro tema y otro acróstico (R-E-G)
    ("human", "Escribe EXACTAMENTE 3 líneas sobre 'regularización'. "
              "Cada línea debe comenzar con R, luego E, luego G. "
              "6-10 palabras por línea. Sin texto extra."),
    ("ai", "R Reduce complejidad para evitar ajustes al ruido\n"
           "E Estabiliza el aprendizaje con penalizaciones adecuadas\n"
           "G Generaliza mejor limitando pesos excesivamente grandes"),
    # Ahora se pide el caso real con el acróstico O-V-E
    ("human", "{instruccion}")
])

print("=== ZERO-SHOT ===")
print((zero_shot | llm_det | parser).invoke({"instruccion": instruccion}))

print("\n=== FEW-SHOT ===")
print((few_shot | llm_det | parser).invoke({"instruccion": instruccion}))

### Role Prompting

Asignarás un rol específico al modelo para influir en el estilo y nivel de detalle de la respuesta.

In [ ]:
# Role prompting
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

role_prompt = ChatPromptTemplate.from_messages([
    ("system", "Actúa como profesor de IA de nivel intermedio. Sé claro, estructurado y usa ejemplos sencillos."),
    ("human", "Explica brevemente qué es el aprendizaje por refuerzo y menciona 2 ejemplos de aplicación."),
])

print((role_prompt | llm | StrOutputParser()).invoke({}))


### CoT (Chain-of-Thought) Prompting

Guiarás al modelo para mostrar pasos intermedios de razonamiento antes de dar una respuesta final.

In [ ]:
# Prompts SIN CoT y CON CoT (Bayes) — nscale-compatible

from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

problema = (
    "En una población, 1% tiene la enfermedad. La prueba tiene 90% de sensibilidad y 90% de especificidad. "
    "Si una persona da positivo, ¿cuál es la probabilidad (en %) de que realmente esté enferma?"
)

parser = StrOutputParser()

# SIN CoT: SOLO porcentaje en una línea (recorta tokens)
endpoint_sin = HuggingFaceEndpoint(
    repo_id=MODEL_ID, task="conversational",
    huggingfacehub_api_token=HF_TOKEN, temperature=0.0, max_new_tokens=6
)
llm_sin = ChatHuggingFace(llm=endpoint_sin)

prompt_sin_cot = ChatPromptTemplate.from_messages([
    ("system", "Devuelve SOLO un número en formato porcentaje (ej: 8.33%). "
               "Sin explicaciones, sin ecuaciones, sin texto extra."),
    ("human", "{q}")
])

# CON CoT: piensa paso a paso y cierra con una línea final
endpoint_con = HuggingFaceEndpoint(
    repo_id=MODEL_ID, task="conversational",
    huggingfacehub_api_token=HF_TOKEN, temperature=0.0, max_new_tokens=256
)
llm_con = ChatHuggingFace(llm=endpoint_con)

prompt_con_cot = ChatPromptTemplate.from_messages([
    ("system", "Tómate tu tiempo y piensa paso a paso usando Bayes. "
               "Al final, da una sola línea con: 'Respuesta final: <n>%'"),
    ("human", "{q}")
])

print("=== SIN CoT ===")
print((prompt_sin_cot | llm_sin | parser).invoke({"q": problema}))

print("\n=== CON CoT ===")
print((prompt_con_cot | llm_con | parser).invoke({"q": problema}))

### Salida estructurada (JSON Output Parser)

Solicitarás al modelo una salida en formato JSON estricto y la validarás con un parser.

In [ ]:
# Salida estructurada (JSON) con output parser
import json
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

json_prompt = ChatPromptTemplate.from_template(
    """
    Eres un asistente que devuelve SIEMPRE JSON válido. Dado un tema, devuelve un objeto con:
    - "titulo": string
    - "puntos_clave": lista de 3 strings
    - "dificultad": uno de ["básico", "intermedio", "avanzado"]
    Responde SOLO con JSON válido sin texto adicional.
    Tema: {tema}
    """
)

json_text = (json_prompt | llm | StrOutputParser()).invoke({"tema": "RAG"})
print(json_text)

data = json.loads(json_text)
assert set(["titulo", "puntos_clave", "dificultad"]).issubset(data.keys())
print("JSON válido con las claves requeridas.")


## Ejercicios: Parte 2

Resuelve los siguientes ejercicios. Modifica prompts y parámetros si es necesario y justifica brevemente tus decisiones (en una celda de texto).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Ejercicio 2.1: Zero-shot vs Few-shot

**Tarea**: explicar "regularización L2" en 3 viñetas claras para un público técnico.

**Pasos**:
1. **Zero-shot**: crea un prompt sin ejemplos y observa el resultado
2. **Few-shot**: agrega 1-2 ejemplos de estilo y compara la salida

**Pregunta guía**: ¿mejoró la precisión o claridad con pocos ejemplos? Justifica.

In [ ]:
from typing import List
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# LLM más determinista
det_endpoint = HuggingFaceEndpoint(
    repo_id=MODEL_ID, task="conversational",
    huggingfacehub_api_token=HF_TOKEN, temperature=0.0, max_new_tokens=128
)
llm_det = ChatHuggingFace(llm=det_endpoint)
parser = StrOutputParser()


def zero_shot_l2(instruccion: str) -> str:
    # TODO: construir prompt zero-shot (3 viñetas) y llamar al LLM
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Eres un asistente útil para un público técnico. Explica el siguiente concepto en 3 viñetas claras."),
        ("human", "Explica '{instruccion}'")
    ])
    chain = prompt | llm_det | parser
    return chain.invoke({"instruccion": instruccion})


def few_shot_l2(instruccion: str) -> str:
    # TODO: construir prompt con 1-2 ejemplos y llamar al LLM
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Eres un asistente útil para un público técnico. Explica el siguiente concepto en 3 viñetas claras."),
        ("human", "Explica 'Dropout'"),
        ("ai", "- Desactiva neuronas aleatoriamente durante el entrenamiento.\n- Reduce el co-adaptación de las neuronas.\n- Ayuda a prevenir el overfitting."),
        ("human", "Explica '{instruccion}'")
    ])
    chain = prompt | llm_det | parser
    return chain.invoke({"instruccion": instruccion})

instruccion_l2 = "regularización L2"

print("\n=== ZERO-SHOT ===\n")
print(zero_shot_l2(instruccion_l2))

print("\n=== FEW-SHOT ===\n")
print(few_shot_l2(instruccion_l2))

### Ejercicio 2.2: Chain-of-Thought (CoT)

**Tarea**: dado un problema de evaluación de modelos, razonar paso a paso y entregar una conclusión final breve.

**Problema sugerido**: "¿Por qué accuracy puede ser engañoso en un dataset muy desbalanceado y qué métrica alternativa usarías?"

**Pista**: pide explícitamente "razona paso a paso y luego da una respuesta final breve en una línea".

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def cot_razonamiento(problema: str) -> str:
    # TODO: pedir "razona paso a paso" y cerrar con una línea final
    endpoint_con = HuggingFaceEndpoint(
        repo_id=MODEL_ID, task="conversational",
        huggingfacehub_api_token=HF_TOKEN, temperature=0.0, max_new_tokens=256
    )
    llm_con = ChatHuggingFace(llm=endpoint_con)

    prompt_con_cot = ChatPromptTemplate.from_messages([
        ("system", "Tómate tu tiempo y piensa paso a paso. Al final, da una sola línea con: 'Respuesta final: <n>'"),
        ("human", "{q}")
    ])
    chain = prompt_con_cot | llm_con | StrOutputParser()
    return chain.invoke({"q": problema})

problema: str = "¿Por qué accuracy puede ser engañoso en un dataset muy desbalanceado y qué métrica alternativa usarías?"
print(cot_razonamiento(problema))

### Ejercicio 2.3: Role Prompting

**Tarea**: explicar el "sesgo de selección" a un equipo de data engineering con ejemplos concisos.

**Rol**: "Actúa como líder técnico de datos; sé pragmático y directo, con viñetas concretas".

**Objetivo**: evaluar cómo cambia el estilo bajo un rol técnico específico.

In [ ]:
# Role prompting
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def explicar_sesgo_seleccion() -> str:
    # TODO: role "líder técnico de datos", 3 viñetas + 1 ejemplo práctico
    role_prompt = ChatPromptTemplate.from_messages([
    ("system", "Actúa como líder técnico de datos; sé pragmático y directo, con viñetas concretas."),
    ("human", "Explica brevemente qué es el sesgo de selección y da un ejemplo práctico."),
    ])

    chain = role_prompt | llm | StrOutputParser()
    return chain.invoke({})

print(explicar_sesgo_seleccion())